In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(color_codes=True)
from sklearn.metrics import mean_squared_error as mse
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import LabelBinarizer,OneHotEncoder
from sklearn.model_selection import GridSearchCV

# 设置jupyter数据框最大显示的列数
pd.set_option('max_columns',100)

In [11]:
train_data = pd.read_csv('../train_data.csv',encoding='gb2312',index_col='id')
# test_data = pd.read_csv('../test_data.csv',encoding='gb2312',index_col='id')
targets = pd.read_csv('../targets.csv',encoding='gb2312',index_col='id')
train = pd.read_csv('../d_train_20180102.csv',encoding='gb2312',index_col='id')
test = pd.read_csv('../d_test_A_20180102.csv',encoding='gb2312',index_col='id')
train,test = train_data.loc[train.index],train_data.loc[test.index]
targets = targets.loc[train.index]
# feature_importance = pd.read_csv('../feature_importance.csv',encoding='gb2312')
test_real_y = pd.read_csv('../d_answer_a_20180128.csv',encoding='gb2312',header=None)

In [18]:
train_data.head()

,性别,年龄,体检日期,*天门冬氨酸氨基转换酶,*丙氨酸氨基转换酶,*碱性磷酸酶,*r-谷氨酰基转换酶,*总蛋白,白蛋白,*球蛋白,白球比例,甘油三酯,总胆固醇,高密度脂蛋白胆固醇,低密度脂蛋白胆固醇,尿素,肌酐,尿酸,乙肝表面抗原,乙肝表面抗体,乙肝e抗原,乙肝e抗体,乙肝核心抗体,白细胞计数,红细胞计数,血红蛋白,红细胞压积,红细胞平均体积,红细胞平均血红蛋白量,红细胞平均血红蛋白浓度,红细胞体积分布宽度,血小板计数,血小板平均体积,血小板体积分布宽度,血小板比积,中性粒细胞%,淋巴细胞%,单核细胞%,嗜酸细胞%,嗜碱细胞%,有无蛋白指标,有无固醇指标,有无血指标,有无细胞指标,有无酶指标,有无尿指标,有无肝指标,缺失值数量,酶_mean,酶_max,...,红细胞体积分布宽度*中性粒细胞%,红细胞体积分布宽度*淋巴细胞%,红细胞体积分布宽度*单核细胞%,红细胞体积分布宽度*嗜酸细胞%,红细胞体积分布宽度*嗜碱细胞%,血小板计数*血小板计数,血小板计数*血小板平均体积,血小板计数*血小板体积分布宽度,血小板计数*血小板比积,血小板计数*中性粒细胞%,血小板计数*淋巴细胞%,血小板计数*单核细胞%,血小板计数*嗜酸细胞%,血小板计数*嗜碱细胞%,血小板平均体积*血小板平均体积,血小板平均体积*血小板体积分布宽度,血小板平均体积*血小板比积,血小板平均体积*中性粒细胞%,血小板平均体积*淋巴细胞%,血小板平均体积*单核细胞%,血小板平均体积*嗜酸细胞%,血小板平均体积*嗜碱细胞%,血小板体积分布宽度*血小板体积分布宽度,血小板体积分布宽度*血小板比积,血小板体积分布宽度*中性粒细胞%,血小板体积分布宽度*淋巴细胞%,血小板体积分布宽度*单核细胞%,血小板体积分布宽度*嗜酸细胞%,血小板体积分布宽度*嗜碱细胞%,血小板比积*血小板比积,血小板比积*中性粒细胞%,血小板比积*淋巴细胞%,血小板比积*单核细胞%,血小板比积*嗜酸细胞%,血小板比积*嗜碱细胞%,中性粒细胞%*中性粒细胞%,中性粒细胞%*淋巴细胞%,中性粒细胞%*单核细胞%,中性粒细胞%*嗜酸细胞%,中性粒细胞%*嗜碱细胞%,淋巴细胞%*淋巴细胞%,淋巴细胞%*单核细胞%,淋巴细胞%*嗜酸细胞%,淋巴细胞%*嗜碱细胞%,单核细胞%*单核细胞%,单核细胞%*嗜酸细胞%,单核细胞%*嗜碱细胞%,嗜酸细胞%*嗜酸细胞%,嗜酸细胞%*嗜碱细胞%,嗜碱细胞%*嗜碱细胞%
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1,41,12/10/2017,24.96,23.10,99.59,20.23,76.88,49.60,27.28,1.82,1.31,4.43,1.37,2.65,5.87,77.25,349.39,0.04,3.22,0.04,1.67,1.69,5.34,5.21,166.1,0.479,91.9,31.9,347.0,12.8,166.0,9.9,17.4,0.164,54.1,34.2,6.5,4.7,0.6,1,1,1,1,1,1,0,5,41.9700,99.59,...,692.48,437.76,83.20,60.16,7.68,27556.0,1643.4,2888.4,27.224,8980.6,5677.2,1079.0,780.2,99.6,98.01,172.26,1.6236,535.59,338.58,64.35,46.53,5.94,302.76,2.8536,941.34,595.08,113.10,81.78,10.44,0.026896,8.8724,5.6088,1.0660,0.7708,0.0984,2926.81,1850.22,351.65,254.27,32.46,1169.64,222.30,160.74,20.52,42.25,30.55,3.90,22.09,2.82,0.36
2,1,41,19/10/2017,24.57,36.25,67.21,79.00,79.43,47.76,31.67,1.51,2.81,4.06,0.93,2.63,5.26,87.12,486.78,0.04,3.22,0.04,1.67,1.69,7.65,5.21,156.0,0.456,87.5,29.9,342.0,13.4,277.0,9.2,10.3,0.260,52.0,36.7,5.8,4.7,0.8,1,1,1,1,1,1,0,5,51.7575,79.00,...,696.80,491.78,77.72,62.98,10.72,76729.0,2548.4,2853.1,72.020,14404.0,10165.9,1606.6,1301.9,221.6,84.64,94.76,2.3920,478.40,337.64,53.36,43.24,7.36,106.09,2.6780,535.60,378.01,59.74,48.41,8.24,0.067600,13.5200,9.5420,1.5080,1.2220,0.2080,2704.00,1908.40,301.60,244.40,41.60,1346.89,212.86,172.49,29.36,33.64,27.26,4.64,22.09,3.76,0.64
3,1,46,26/10/2017,20.82,15.23,63.69,38.17,86.23,48.00,38.23,1.26,0.99,4.13,1.64,2.01,4.77,78.19,452.07,0.01,0.02,0.01,1.37,1.07,4.60,4.76,148.8,0.438,91.9,31.3,340.0,13.0,241.0,8.3,16.6,0.199,48.1,40.3,7.7,3.2,0.8,1,1,1,1,1,1,1,0,34.4775,63.69,...,625.30,523.90,100.10,41.60,10.40,58081.0,2000.3,4000.6,47.959,11592.1,9712.3,1855.7,771.2,192.8,68.89,137.78,1.6517,399.23,334.49,63.91,26.56,6.64,275.56,3.3034,798.46,668.98,127.82,53.12,13.28,0.039601,9.5719,8.0197,1.5323,0.6368,0.1592,2313.61,1938.43,370.37,153.92,38.48,1624.09,310.31,128.96,32.24,59.29,24.64,6.16,10.24,2.56,0.64
4,0,22,25/10/2017,14.99,10.59,74.08,20.22,70.98,44.02,26.96,1.63,1.06,6.89,1.43,5.04,4.23,61.46,368.85,0.04,3.22,0.04,1.67,1.69,9.36,4.29,137.0,0.403,93.9,31.9,340.0,12.6,252.0,10.3,10.8,0.260,41.7,46.5,6.7,4.6,0.5,1,1,1,1,1,1,0,5,29.9700,74.08,...,525.42,585.90,84.42,57.96,6.30,63504.0,2595.6,2721.6,65.520,10508.4,11718.0,1688.4,1159.2,126.0,106.09,111.24,2.6780,429.51,478.95,69.01,47.38,5.15,116.64,2.8080,450.36,502.20,72.36,49.68,5.40,0.067600,10.8420,12.0900,1.7420,1.1960,0.1300,1738.89,1939.05,279.39,191.82,20.85,2162.25,311.55,213.90,23.25,44.89,30.82,3.35,21.16,2.30,0.25
5,0,48,26/10/2017,20.07,14.78,75.79,22.72,78.05,41.83,36.22,1.15,0.97,5.37,1.27,3.65,4.86,77.18,341.67,0.04,3.22,0.04,1.67,1.69,5.07,5.15,106.0,0.354,68.7,20.6,299.0,16.6,316.0,11.1,14.0,0.350,56.6,33.1,9.1,0.6,0.6,1,1,1,1,1,0,0,8,33.3400,75.79,...,939.56,549.46,151.06,9.96,9.96,99856.0,3507.6,4424.0,110.600,17885.6,10459.6,2875.6,189.6,189.6,123.21,155.40,3.8850,628.26,367.41,101.01,6.66,6.66,196.00,4.9000,792.40,463.40,127.40,8.40,8.40,0.122500,19.8100,11.5850,3.1850,0.2100,0.2100,3203.56,1873.46,515.06,33.96,33.96,1095.61,301.21,19

In [20]:
from xgboost.sklearn import XGBRegressor
# from sklearn.metrics import f1_score
from sklearn.cross_validation import train_test_split
from sklearn.model_selection import cross_val_score
import xgboost as xgb
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import ElasticNet,LinearRegression
from sklearn.model_selection import cross_val_score


#  数据集
# most_important = feature_importance['feature'].loc[:50]
# poly = PolynomialFeatures(2)
# train_data_most_important = pd.DataFrame(poly.fit_transform(train_data[most_important]),index=train_data.index)
# train_data = pd.concat([train_data.drop(most_important,axis=1),train_data_most_important],axis=1)
# test_data_most_important = pd.DataFrame(poly.transform(test_data[most_important]),index=test_data.index)
# test_data = pd.concat([test_data.drop(most_important,axis=1),test_data_most_important],axis=1)
#training xgboost
train_data.drop(['体检日期'],axis=1,inplace=True)
test_data.drop(['体检日期'],axis=1,inplace=True)
dtrain = xgb.DMatrix(train_data,label=targets)
dtest = xgb.DMatrix(test_data)
# 交叉验证
params={'booster':'gbtree',
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'max_depth':3,
    'lambda':200,
    'subsample':0.75,
    'colsample_bytree':0.7,
    'eta': 0.006,#0.002
    'seed':1024,
    'nthread':4,
    'reg_alpha':0.1
    }

# 设置参数
watchlist  = [(dtrain,'train')]

model = xgb.train(params,dtrain,num_boost_round=3000,evals=watchlist,early_stopping_rounds=50)

pre_y = model.predict(dtest)


[0]	train-rmse:5.34337
Will train until train-rmse hasn't improved in 50 rounds.
[1]	train-rmse:5.31513
[2]	train-rmse:5.28707
[3]	train-rmse:5.25905
[4]	train-rmse:5.23128
[5]	train-rmse:5.20365
[6]	train-rmse:5.17628
[7]	train-rmse:5.14907
[8]	train-rmse:5.12193
[9]	train-rmse:5.09509
[10]	train-rmse:5.06834
[11]	train-rmse:5.04175
[12]	train-rmse:5.01537
[13]	train-rmse:4.98911
[14]	train-rmse:4.9632
[15]	train-rmse:4.93729
[16]	train-rmse:4.91144
[17]	train-rmse:4.88594
[18]	train-rmse:4.86055
[19]	train-rmse:4.83528
[20]	train-rmse:4.81022
[21]	train-rmse:4.78533
[22]	train-rmse:4.76049
[23]	train-rmse:4.73584
[24]	train-rmse:4.7115
[25]	train-rmse:4.68727
[26]	train-rmse:4.66322
[27]	train-rmse:4.63924
[28]	train-rmse:4.61548
[29]	train-rmse:4.59183
[30]	train-rmse:4.56829
[31]	train-rmse:4.54481
[32]	train-rmse:4.52162
[33]	train-rmse:4.49845
[34]	train-rmse:4.4755
[35]	train-rmse:4.4527
[36]	train-rmse:4.43001
[37]	train-rmse:4.40748
[38]	train-rmse:4.38507
[39]	train-rmse:4.36

[332]	train-rmse:1.66797
[333]	train-rmse:1.66579
[334]	train-rmse:1.66361
[335]	train-rmse:1.66142
[336]	train-rmse:1.65927
[337]	train-rmse:1.65716
[338]	train-rmse:1.65504
[339]	train-rmse:1.65296
[340]	train-rmse:1.65088
[341]	train-rmse:1.64884
[342]	train-rmse:1.64679
[343]	train-rmse:1.64478
[344]	train-rmse:1.64274
[345]	train-rmse:1.64076
[346]	train-rmse:1.63882
[347]	train-rmse:1.6369
[348]	train-rmse:1.63496
[349]	train-rmse:1.63303
[350]	train-rmse:1.6311
[351]	train-rmse:1.62919
[352]	train-rmse:1.62728
[353]	train-rmse:1.62546
[354]	train-rmse:1.62363
[355]	train-rmse:1.6218
[356]	train-rmse:1.62004
[357]	train-rmse:1.61825
[358]	train-rmse:1.61648
[359]	train-rmse:1.61478
[360]	train-rmse:1.6131
[361]	train-rmse:1.61137
[362]	train-rmse:1.60965
[363]	train-rmse:1.60798
[364]	train-rmse:1.60631
[365]	train-rmse:1.6046
[366]	train-rmse:1.60297
[367]	train-rmse:1.60133
[368]	train-rmse:1.5997
[369]	train-rmse:1.59809
[370]	train-rmse:1.59653
[371]	train-rmse:1.59494
[372]	

[662]	train-rmse:1.39939
[663]	train-rmse:1.3991
[664]	train-rmse:1.39883
[665]	train-rmse:1.39854
[666]	train-rmse:1.39825
[667]	train-rmse:1.39797
[668]	train-rmse:1.39769
[669]	train-rmse:1.39742
[670]	train-rmse:1.39712
[671]	train-rmse:1.39687
[672]	train-rmse:1.39659
[673]	train-rmse:1.39629
[674]	train-rmse:1.39601
[675]	train-rmse:1.39575
[676]	train-rmse:1.3955
[677]	train-rmse:1.39523
[678]	train-rmse:1.39498
[679]	train-rmse:1.39469
[680]	train-rmse:1.39442
[681]	train-rmse:1.39415
[682]	train-rmse:1.39392
[683]	train-rmse:1.39368
[684]	train-rmse:1.39341
[685]	train-rmse:1.39314
[686]	train-rmse:1.39288
[687]	train-rmse:1.39265
[688]	train-rmse:1.39234
[689]	train-rmse:1.3921
[690]	train-rmse:1.39184
[691]	train-rmse:1.39158
[692]	train-rmse:1.39134
[693]	train-rmse:1.39109
[694]	train-rmse:1.39081
[695]	train-rmse:1.39055
[696]	train-rmse:1.39029
[697]	train-rmse:1.39002
[698]	train-rmse:1.38978
[699]	train-rmse:1.3895
[700]	train-rmse:1.38923
[701]	train-rmse:1.38902
[702

[992]	train-rmse:1.33415
[993]	train-rmse:1.33399
[994]	train-rmse:1.33383
[995]	train-rmse:1.33366
[996]	train-rmse:1.3335
[997]	train-rmse:1.33337
[998]	train-rmse:1.3332
[999]	train-rmse:1.33307
[1000]	train-rmse:1.33292
[1001]	train-rmse:1.33279
[1002]	train-rmse:1.33263
[1003]	train-rmse:1.33246
[1004]	train-rmse:1.33232
[1005]	train-rmse:1.33217
[1006]	train-rmse:1.33202
[1007]	train-rmse:1.33189
[1008]	train-rmse:1.33173
[1009]	train-rmse:1.33159
[1010]	train-rmse:1.33146
[1011]	train-rmse:1.33132
[1012]	train-rmse:1.33119
[1013]	train-rmse:1.33104
[1014]	train-rmse:1.3309
[1015]	train-rmse:1.33075
[1016]	train-rmse:1.33062
[1017]	train-rmse:1.33049
[1018]	train-rmse:1.33032
[1019]	train-rmse:1.33018
[1020]	train-rmse:1.33003
[1021]	train-rmse:1.32988
[1022]	train-rmse:1.32973
[1023]	train-rmse:1.3296
[1024]	train-rmse:1.32945
[1025]	train-rmse:1.32931
[1026]	train-rmse:1.32915
[1027]	train-rmse:1.32901
[1028]	train-rmse:1.32888
[1029]	train-rmse:1.32873
[1030]	train-rmse:1.3286

[1309]	train-rmse:1.2944
[1310]	train-rmse:1.29427
[1311]	train-rmse:1.29413
[1312]	train-rmse:1.29404
[1313]	train-rmse:1.29394
[1314]	train-rmse:1.29383
[1315]	train-rmse:1.29374
[1316]	train-rmse:1.29365
[1317]	train-rmse:1.29354
[1318]	train-rmse:1.29346
[1319]	train-rmse:1.29333
[1320]	train-rmse:1.29321
[1321]	train-rmse:1.29313
[1322]	train-rmse:1.29305
[1323]	train-rmse:1.29295
[1324]	train-rmse:1.29287
[1325]	train-rmse:1.29276
[1326]	train-rmse:1.29263
[1327]	train-rmse:1.29249
[1328]	train-rmse:1.29238
[1329]	train-rmse:1.29226
[1330]	train-rmse:1.29217
[1331]	train-rmse:1.29207
[1332]	train-rmse:1.29195
[1333]	train-rmse:1.29185
[1334]	train-rmse:1.29173
[1335]	train-rmse:1.29163
[1336]	train-rmse:1.29152
[1337]	train-rmse:1.29143
[1338]	train-rmse:1.29134
[1339]	train-rmse:1.29121
[1340]	train-rmse:1.29109
[1341]	train-rmse:1.29099
[1342]	train-rmse:1.29086
[1343]	train-rmse:1.29075
[1344]	train-rmse:1.29063
[1345]	train-rmse:1.29052
[1346]	train-rmse:1.29044
[1347]	train-

[1625]	train-rmse:1.26388
[1626]	train-rmse:1.26381
[1627]	train-rmse:1.26375
[1628]	train-rmse:1.26366
[1629]	train-rmse:1.26358
[1630]	train-rmse:1.26347
[1631]	train-rmse:1.26339
[1632]	train-rmse:1.26329
[1633]	train-rmse:1.2632
[1634]	train-rmse:1.26312
[1635]	train-rmse:1.26302
[1636]	train-rmse:1.26292
[1637]	train-rmse:1.26283
[1638]	train-rmse:1.26275
[1639]	train-rmse:1.26268
[1640]	train-rmse:1.26256
[1641]	train-rmse:1.26248
[1642]	train-rmse:1.26241
[1643]	train-rmse:1.26232
[1644]	train-rmse:1.26224
[1645]	train-rmse:1.26214
[1646]	train-rmse:1.26206
[1647]	train-rmse:1.26196
[1648]	train-rmse:1.26189
[1649]	train-rmse:1.26179
[1650]	train-rmse:1.26172
[1651]	train-rmse:1.26162
[1652]	train-rmse:1.26154
[1653]	train-rmse:1.26149
[1654]	train-rmse:1.26137
[1655]	train-rmse:1.26132
[1656]	train-rmse:1.26125
[1657]	train-rmse:1.26119
[1658]	train-rmse:1.26113
[1659]	train-rmse:1.26108
[1660]	train-rmse:1.26098
[1661]	train-rmse:1.2609
[1662]	train-rmse:1.26083
[1663]	train-r

[1942]	train-rmse:1.23838
[1943]	train-rmse:1.23828
[1944]	train-rmse:1.23822
[1945]	train-rmse:1.23813
[1946]	train-rmse:1.23802
[1947]	train-rmse:1.23795
[1948]	train-rmse:1.23788
[1949]	train-rmse:1.2378
[1950]	train-rmse:1.23771
[1951]	train-rmse:1.23763
[1952]	train-rmse:1.23755
[1953]	train-rmse:1.23749
[1954]	train-rmse:1.23743
[1955]	train-rmse:1.23734
[1956]	train-rmse:1.23726
[1957]	train-rmse:1.2372
[1958]	train-rmse:1.23713
[1959]	train-rmse:1.23706
[1960]	train-rmse:1.237
[1961]	train-rmse:1.2369
[1962]	train-rmse:1.23684
[1963]	train-rmse:1.23676
[1964]	train-rmse:1.23669
[1965]	train-rmse:1.23663
[1966]	train-rmse:1.23654
[1967]	train-rmse:1.23645
[1968]	train-rmse:1.23635
[1969]	train-rmse:1.23628
[1970]	train-rmse:1.2362
[1971]	train-rmse:1.2361
[1972]	train-rmse:1.23606
[1973]	train-rmse:1.23599
[1974]	train-rmse:1.23594
[1975]	train-rmse:1.23584
[1976]	train-rmse:1.23575
[1977]	train-rmse:1.23565
[1978]	train-rmse:1.23556
[1979]	train-rmse:1.23548
[1980]	train-rmse:1

[2259]	train-rmse:1.21611
[2260]	train-rmse:1.21605
[2261]	train-rmse:1.21598
[2262]	train-rmse:1.21593
[2263]	train-rmse:1.21588
[2264]	train-rmse:1.2158
[2265]	train-rmse:1.21572
[2266]	train-rmse:1.21563
[2267]	train-rmse:1.21557
[2268]	train-rmse:1.2155
[2269]	train-rmse:1.2154
[2270]	train-rmse:1.21533
[2271]	train-rmse:1.21526
[2272]	train-rmse:1.21517
[2273]	train-rmse:1.21509
[2274]	train-rmse:1.21503
[2275]	train-rmse:1.21498
[2276]	train-rmse:1.21492
[2277]	train-rmse:1.21484
[2278]	train-rmse:1.21475
[2279]	train-rmse:1.21468
[2280]	train-rmse:1.21463
[2281]	train-rmse:1.21457
[2282]	train-rmse:1.2145
[2283]	train-rmse:1.2144
[2284]	train-rmse:1.21433
[2285]	train-rmse:1.21427
[2286]	train-rmse:1.2142
[2287]	train-rmse:1.21416
[2288]	train-rmse:1.21409
[2289]	train-rmse:1.21404
[2290]	train-rmse:1.21396
[2291]	train-rmse:1.21388
[2292]	train-rmse:1.21382
[2293]	train-rmse:1.21378
[2294]	train-rmse:1.21373
[2295]	train-rmse:1.21365
[2296]	train-rmse:1.2136
[2297]	train-rmse:1

[2576]	train-rmse:1.19546
[2577]	train-rmse:1.19541
[2578]	train-rmse:1.19534
[2579]	train-rmse:1.19529
[2580]	train-rmse:1.19524
[2581]	train-rmse:1.19515
[2582]	train-rmse:1.19507
[2583]	train-rmse:1.19502
[2584]	train-rmse:1.19497
[2585]	train-rmse:1.19491
[2586]	train-rmse:1.19484
[2587]	train-rmse:1.19478
[2588]	train-rmse:1.19473
[2589]	train-rmse:1.19465
[2590]	train-rmse:1.19457
[2591]	train-rmse:1.19453
[2592]	train-rmse:1.19449
[2593]	train-rmse:1.19444
[2594]	train-rmse:1.19438
[2595]	train-rmse:1.19433
[2596]	train-rmse:1.19424
[2597]	train-rmse:1.1942
[2598]	train-rmse:1.19415
[2599]	train-rmse:1.19408
[2600]	train-rmse:1.19403
[2601]	train-rmse:1.19398
[2602]	train-rmse:1.19392
[2603]	train-rmse:1.19385
[2604]	train-rmse:1.1938
[2605]	train-rmse:1.19376
[2606]	train-rmse:1.1937
[2607]	train-rmse:1.19366
[2608]	train-rmse:1.19361
[2609]	train-rmse:1.19353
[2610]	train-rmse:1.19347
[2611]	train-rmse:1.19342
[2612]	train-rmse:1.19335
[2613]	train-rmse:1.1933
[2614]	train-rms

[2892]	train-rmse:1.17653
[2893]	train-rmse:1.17642
[2894]	train-rmse:1.17635
[2895]	train-rmse:1.1763
[2896]	train-rmse:1.17623
[2897]	train-rmse:1.17617
[2898]	train-rmse:1.17612
[2899]	train-rmse:1.17607
[2900]	train-rmse:1.176
[2901]	train-rmse:1.17594
[2902]	train-rmse:1.17588
[2903]	train-rmse:1.17581
[2904]	train-rmse:1.17576
[2905]	train-rmse:1.1757
[2906]	train-rmse:1.17563
[2907]	train-rmse:1.17554
[2908]	train-rmse:1.1755
[2909]	train-rmse:1.17543
[2910]	train-rmse:1.17537
[2911]	train-rmse:1.17529
[2912]	train-rmse:1.17523
[2913]	train-rmse:1.17519
[2914]	train-rmse:1.17512
[2915]	train-rmse:1.17506
[2916]	train-rmse:1.17502
[2917]	train-rmse:1.17494
[2918]	train-rmse:1.1749
[2919]	train-rmse:1.17485
[2920]	train-rmse:1.17481
[2921]	train-rmse:1.17473
[2922]	train-rmse:1.17466
[2923]	train-rmse:1.1746
[2924]	train-rmse:1.17456
[2925]	train-rmse:1.17451
[2926]	train-rmse:1.17445
[2927]	train-rmse:1.17437
[2928]	train-rmse:1.17432
[2929]	train-rmse:1.17427
[2930]	train-rmse:1

In [52]:
print(mse(test_real_y,pre_y))

1.68255054944


In [57]:
pre_y = model.predict(dtest)
print(mse(test_real_y,pre_y))

1.67513493916


In [62]:
print(mse(test_real_y,pre_y))

1.65532476475


In [69]:
print(mse(test_real_y,pre_y))

1.6536369937


In [73]:
print(mse(test_real_y,pre_y))

1.65513232487


In [21]:
results = pd.DataFrame(pre_y)
results.to_csv('../results.csv',index=False,header=None)

In [5]:
# results

In [ ]:
from sklearn.linear_model import ElasticNet,LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.cross_validation import train_test_split

# train_x,test_x,train_y,test_y = train_test_split(train_data,targets,test_size=0.5)
enet = ElasticNet(alpha=1, l1_ratio=0,normalize=True,tol=0.01)
# train.drop(['体检日期'],axis=1,inplace=True)
# test.drop(['体检日期'],axis=1,inplace=True)
pre_y = enet.fit(train, targets).predict(test)
# 预测
pre_y[pre_y >20] = 20
pre_y[pre_y<3] = 3
print(mse(test_real_y,pre_y))
pre_train_y = enet.predict(train)
print(mse(pre_train_y,targets))

In [10]:
targets.shape

(6642, 1)

In [ ]:
df.apply(lambda x : np.sort(x)[-2],axis=0)